# Mentahan

In [2]:
from google.colab import drive
# When you run this cell, a prompt will appear asking you to authorize Google Drive access.
# Click the link, select your Google account, and allow the necessary permissions.
# Then, copy the authorization code provided and paste it back into the input box in Colab.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd

def read_signal_from_txt(file_path):
    with open(file_path, 'r') as f:
        data = f.readlines()
    data = [float(line.strip()) for line in data]
    return data

def load_dataset_from_folder(folder_path, label):
    rows = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            signal = read_signal_from_txt(file_path)
            if len(signal) == 4097:
                rows.append([label] + signal)
    return rows

# Ganti path ke lokasi dataset kamu
base_path = "C:/Users/syahla/Downloads/z"

# Folder-folder kategori EEG
categories = {
    'Z': 0,  # non-epileptic
    'S': 1   # epileptic
}

all_data = []

for folder, label in categories.items():
    folder_path = os.path.join(base_path, folder)
    data = load_dataset_from_folder(folder_path, label)
    all_data.extend(data)

# Simpan ke CSV
df = pd.DataFrame(all_data)
df.to_csv("EEG_epilepsy_dataset.csv", index=False, header=["label"] + [f"v{i}" for i in range(1, 4098)])

print("Konversi selesai. Dataset disimpan di EEG_epilepsy_dataset.csv")


In [ ]:
import scipy.io
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# -------------------------------
# 1. Load Data from All 5 Sets
# -------------------------------

def load_data_from_mat(file_path, label):
    mat = scipy.io.loadmat(file_path)
    data = mat['data']  # Data assumed to be under 'data' key
    return data, np.full((data.shape[0],), label)

# Path to directory containing the 5 .mat files
data_dir = '/content/drive/MyDrive/dataset eeg'  # Ubah sesuai direktori kamu

# File names (gunakan nama file asli seperti "Z.mat", "O.mat", dst.)
file_labels = {
    'Z.mat': 0,  # non-seizure
    'O.mat': 0,
    'N.mat': 0,
    'F.mat': 0,
    'S.mat': 1   # seizure
}

X, y = [], []

for fname, label in file_labels.items():
    data, labels = load_data_from_mat(os.path.join(data_dir, fname), label)
    X.append(data)
    y.append(labels)

X = np.vstack(X)
y = np.hstack(y)

print("Data shape:", X.shape)  # (500, 4097)
print("Labels shape:", y.shape)

# -------------------------------
# 2. Preprocessing
# -------------------------------

# Normalisasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape untuk CNN input [samples, time steps, channels]
X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# -------------------------------
# 3. Build CNN Model
# -------------------------------

model = Sequential([
    Conv1D(32, kernel_size=5, activation='relu', input_shape=(4097, 1)),
    MaxPooling1D(pool_size=2),

    Conv1D(64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# -------------------------------
# 4. Train Model
# -------------------------------

history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.2)

# -------------------------------
# 5. Evaluate Model
# -------------------------------

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

# -------------------------------
# 6. Visualisasi
# -------------------------------

plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title("CNN Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
